In [18]:
import pandas as pd
import os
from glob import glob

In [2]:
#Reading in multiple csv files of Citi Bike Trip History Logs (2018 - 2021)
filenames = glob('JC*.csv')
citibikes = pd.concat((pd.read_csv(f) for f in filenames), ignore_index = True)

#### Data Cleaning 

In [3]:
citibikes.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,156,2020-10-01 00:02:40.2600,2020-10-01 00:05:17.0140,3186,Grove St PATH,40.719586,-74.043117,3270,Jersey & 6th St,40.725289,-74.045572,42293,Subscriber,1996,1
1,143,2020-10-01 00:02:53.5020,2020-10-01 00:05:16.9650,3186,Grove St PATH,40.719586,-74.043117,3270,Jersey & 6th St,40.725289,-74.045572,44740,Subscriber,1998,1
2,305,2020-10-01 00:04:31.3160,2020-10-01 00:09:36.3840,3273,Manila & 1st,40.721651,-74.042884,3269,Brunswick & 6th,40.726012,-74.050389,41369,Subscriber,1988,2
3,1097,2020-10-01 00:05:18.3680,2020-10-01 00:23:36.1240,3199,Newport Pkwy,40.728745,-74.032108,3281,Leonard Gordon Park,40.745910,-74.057271,42285,Customer,2001,1
4,277,2020-10-01 00:09:17.5990,2020-10-01 00:13:55.5150,3199,Newport Pkwy,40.728745,-74.032108,3638,Washington St,40.724294,-74.035483,45231,Subscriber,1965,1


In [4]:
#Ensuring that all column names are meaningful and descriptive
citibikes.columns

Index(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'birth year', 'gender'],
      dtype='object')

In [5]:
#Checking for nill values
citibikes.isnull().values.any()

False

In [6]:
citibikes.dtypes

tripduration                 int64
starttime                   object
stoptime                    object
start station id             int64
start station name          object
start station latitude     float64
start station longitude    float64
end station id               int64
end station name            object
end station latitude       float64
end station longitude      float64
bikeid                       int64
usertype                    object
birth year                   int64
gender                       int64
dtype: object

In [7]:
#Converting dates and time to pandas datetime type
citibikes['starttime'] = pd.to_datetime(citibikes['starttime'], format='%Y-%m-%d %H:%M:%S.%f')
citibikes['stoptime'] = pd.to_datetime(citibikes['stoptime'], format='%Y-%m-%d %H:%M:%S.%f')
citibikes['birth year'] = pd.to_datetime(citibikes['birth year'], format='%Y')

In [8]:
#Extract trip year to a different column for timeseries graph later
citibikes['trip year'] = citibikes['starttime'].map(lambda x: x.strftime('%Y'))

In [9]:
#Confirming starttime, stoptime and birth year are all in pandas datetime type
citibikes.dtypes

tripduration                        int64
starttime                  datetime64[ns]
stoptime                   datetime64[ns]
start station id                    int64
start station name                 object
start station latitude            float64
start station longitude           float64
end station id                      int64
end station name                   object
end station latitude              float64
end station longitude             float64
bikeid                              int64
usertype                           object
birth year                 datetime64[ns]
gender                              int64
trip year                          object
dtype: object

In [10]:
citibikes.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,trip year
0,156,2020-10-01 00:02:40.260,2020-10-01 00:05:17.014,3186,Grove St PATH,40.719586,-74.043117,3270,Jersey & 6th St,40.725289,-74.045572,42293,Subscriber,1996-01-01,1,2020
1,143,2020-10-01 00:02:53.502,2020-10-01 00:05:16.965,3186,Grove St PATH,40.719586,-74.043117,3270,Jersey & 6th St,40.725289,-74.045572,44740,Subscriber,1998-01-01,1,2020
2,305,2020-10-01 00:04:31.316,2020-10-01 00:09:36.384,3273,Manila & 1st,40.721651,-74.042884,3269,Brunswick & 6th,40.726012,-74.050389,41369,Subscriber,1988-01-01,2,2020
3,1097,2020-10-01 00:05:18.368,2020-10-01 00:23:36.124,3199,Newport Pkwy,40.728745,-74.032108,3281,Leonard Gordon Park,40.745910,-74.057271,42285,Customer,2001-01-01,1,2020
4,277,2020-10-01 00:09:17.599,2020-10-01 00:13:55.515,3199,Newport Pkwy,40.728745,-74.032108,3638,Washington St,40.724294,-74.035483,45231,Subscriber,1965-01-01,1,2020


In [11]:
citibikes['trip year'].unique()

array(['2020', '2019', '2018', '2021'], dtype=object)

In [12]:
#Replacing the gender column with the citibike key for gender - > (Zero=unknown; 1=male; 2=female)
citibikes['gender']= citibikes['gender'].replace({1 : 'Male', 2: 'Female', 0: 'Unknown' })

In [13]:
#Checking that all the keys are replaced 
citibikes['gender'].unique()

array(['Male', 'Female', 'Unknown'], dtype=object)

In [14]:
#Sorting citibikes dataframe by starttime 
citibikes = citibikes.sort_values(by=['starttime']).reset_index(drop = True)

In [15]:
citibikes.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,trip year
0,132,2018-01-01 00:01:45.814,2018-01-01 00:03:58.475,3186,Grove St PATH,40.719586,-74.043117,3211,Newark Ave,40.721525,-74.046305,29590,Subscriber,1964-01-01,Male,2018
1,560,2018-01-01 01:27:17.272,2018-01-01 01:36:37.841,3276,Marin Light Rail,40.714584,-74.042817,3269,Brunswick & 6th,40.726012,-74.050389,29613,Subscriber,1989-01-01,Female,2018
2,294,2018-01-01 01:29:03.020,2018-01-01 01:33:57.899,3195,Sip Ave,40.730743,-74.063784,3225,Baldwin at Montgomery,40.723659,-74.064194,31940,Subscriber,1994-01-01,Male,2018
3,197,2018-01-01 01:59:31.746,2018-01-01 02:02:48.996,3211,Newark Ave,40.721525,-74.046305,3278,Monmouth and 6th,40.725685,-74.048790,31949,Subscriber,1964-01-01,Male,2018
4,932,2018-01-01 02:06:17.541,2018-01-01 02:21:50.027,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,-74.032108,31929,Subscriber,1992-01-01,Male,2018


In [16]:
# Save citibikes data into a csv file
citibikes.to_csv("citibikes.csv", index = False)